In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep
import re
import numpy as np
import requests
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import smtplib
from email.message import EmailMessage
import os

## Note: safe to ensure relevant libraries are in the pricing_runner.yml to ensure the libraries are installed before use

*Barchart extraction method*

In [ ]:
# Month code to name mapping used in futures contracts (standardisation)
month_map = {
    "F": "Jan", "G": "Feb", "H": "Mar", "J": "Apr", "K": "May", "M": "Jun",
    "N": "Jul", "Q": "Aug", "U": "Sep", "V": "Oct", "X": "Nov", "Z": "Dec"
}
month_codes = list(month_map)  # ['F', 'G', ..., 'Z']

# Parse symbols
# default settings
# Generate futures contract symbols for a given base code from start_date forward.
def parse_barchart_symbols(base_code: str, start_date: datetime, years_forward: int):

    # Args:
    #     base_code (str): e.g., "NG", "QA", "INK"
    #     start_date (datetime): Starting date (usually today)
    #     years_forward (int): Number of years into the future to pull

    #Returns: List of contract symbols like ["NGU25", "NGV25", ...]
    symbols = []
    current = start_date.replace(day=1)
    end_date = current + relativedelta(years=years_forward)
    end_year = end_date.year

    while current.year < end_year:
        y = str(current.year)[-2:]
        m_code = list(month_map.keys())[current.month - 1]
        symbols.append(f"{base_code}{m_code}{y}")
        current += relativedelta(months=1)

    # Add the **full final year** (Jan to Dec)
    for m_idx, m_code in enumerate(month_map.keys()):
        symbols.append(f"{base_code}{m_code}{str(end_year)[-2:]}")

    return symbols

## Fetch futures prices from Barchart symbol pages.
def fetch_barchart_prices(symbols, base_url="https://www.barchart.com/futures/quotes/{}/overview"):
    # Args:
    #     symbols (list): List of contract symbols like ["NGU25", "NGV25"]
    #     base_url (str): URL format string with `{}` for symbol insertion

    # Returns:
    #     pd.DataFrame: Cleaned dataframe with columns ["symbol", "last_price"]
    headers = {"User-Agent": "Mozilla/5.0"}
    records = []

    for symbol in symbols:
        url = base_url.format(symbol)
        try:
            # Throttle request rate (Barchart rate-limits aggressively)
            sleep(0.5) # remove if it becomes too slow

            resp = requests.get(url, headers=headers, timeout=10)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, "html.parser")

            # Find the container with init() JSON
            div = soup.find("div", class_="symbol-header-info")
            if not div:
                print(f"[WARN] Missing div for {symbol}")
                continue

            ng_init = div.get("data-ng-init")
            match = re.search(r'init\((\{.*?\})\)', ng_init)
            if not match:
                print(f"[WARN] No JSON init block found for {symbol}")
                continue

            data = json.loads(match.group(1).replace(r'\/', '/'))
            raw_price = data.get("lastPrice")

            if raw_price in [None, "N/A", "-"]:
                print(f"[SKIP] Invalid price for {symbol}")
                continue

            clean_price = re.sub(r"[^\d.]+$", "", str(raw_price))
            records.append({
                "symbol": symbol,
                "last_price": round(float(clean_price), 4) if clean_price else None
            })

        except Exception as e:
            print(f"[ERROR] Failed to fetch {symbol}: {e}")
            continue

    if not records:
        print("[ERROR] No valid prices fetched.")
        return pd.DataFrame(columns=["symbol", "last_price"])

    return pd.DataFrame(records)

## Abit of housekeeping/cleaning at the end to tie it all together
def clean_and_format_df(df_raw, label):
    df = df_raw.copy()

    # Extract month code and map to month name
    df["month_code"] = df["symbol"].str.extract(r'[A-Z]{2,3}([FGHJKMNQUVXZ])\d{2}')
    df["month"] = df["month_code"].map(month_map)

    # Extract year
    df["year"] = 2000 + df["symbol"].str.extract(r'(\d{2})$').astype(int)

    # Dynamically map current month and year for cash contracts
    cash_mask = df["symbol"].str.endswith("Y00")
    if cash_mask.any():
        df.loc[cash_mask, "month"] = datetime.today().strftime("%b")
        df.loc[cash_mask, "year"] = datetime.today().year

    # Label construction: Cash for Y00 contracts, Month'YY otherwise
    df["label"] = np.where(
        cash_mask,
        "Cash",
        df["month"].str[:3] + "'" + df["year"].astype(str).str[-2:]
    )
    
    df = df.rename(columns={"last_price": label})
    return df[["label", label]].reset_index(drop=True)

*Pulling the individual components/tickers from barchart.com*

In [3]:
### Part to start pulling/scraping the data from barchart.com
today = datetime.today()
start_date = today

# Pull configuration for each commodity
commodity_config = {
    'NG': {'label': "Henry Hub ($/MMBtu)", 'base_code': 'NG', 'years': 12, 'has_cash': True, 'cash_symbol': 'NGY00'},
    'QA': {'label': "Brent ($/bbl)", 'base_code': 'QA', 'years': 8, 'has_cash': True, 'cash_symbol': 'QAY00'},
    'INK': {'label': "TTF ($/MMBtu)", 'base_code': 'INK', 'years': 3, 'has_cash': False},
    'JKM': {'label': "JKM ($/MMBtu)", 'base_code': 'JKM', 'years': 5, 'has_cash': False},
    'NF': {'label': "NBP (p/th)", 'base_code': 'NF', 'years': 7, 'has_cash': False}
}

# Container to store clean DFs
commodity_data = {}

In [4]:
for code, cfg in commodity_config.items():
    symbols = []

    # Add contract if it does exist (Note that Brent and Henry have cash contracts)
    if cfg.get('has_cash'):
        symbols.append(cfg['cash_symbol'])

    # Add futures symbols
    symbols += parse_barchart_symbols(
        base_code=cfg['base_code'],
        start_date=start_date,
        years_forward=cfg['years']
    )

    # Fetch prices
    raw_df = fetch_barchart_prices(symbols)

    # Clean and format
    clean_df = clean_and_format_df(raw_df, cfg['label'])

    # Drop current front-month & label dynamically
    front_month_label = today.strftime("%b'%y")
    clean_df = clean_df[clean_df["label"] != front_month_label]

    commodity_data[code] = clean_df

# cross checker - ensures the right tickers come out
for code, df in commodity_data.items():
    if not df.empty:
        print(f"[INFO] {code} curve ends at:", df['label'].iloc[-1])

[ERROR] Failed to fetch QAJ33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAJ33/overview
[ERROR] Failed to fetch QAK33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAK33/overview
[ERROR] Failed to fetch QAM33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAM33/overview
[ERROR] Failed to fetch QAN33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAN33/overview
[ERROR] Failed to fetch QAQ33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAQ33/overview
[ERROR] Failed to fetch QAU33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAU33/overview
[ERROR] Failed to fetch QAV33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAV33/overview
[ERROR] Failed to fetch QAX33: 404 Client Error: Not Found for url: https://www.barchart.com/futures/quotes/QAX33/overview
[ERROR] Failed t

In [6]:
# Individual flat prices we wish to obtain from barchart
df_ng = commodity_data["NG"] # Henrys
df_qa = commodity_data["QA"] # Brent
df_ink = commodity_data["INK"] # TTF
df_jkm = commodity_data["JKM"] # JKM
df_nbp = commodity_data["NF"] # NBP curve

*Pulling Cable fwd rates - needed for accurate computation of the NBP curve in $/mmbtu*

* The EOD curves we get from brokers are a lazy in their derivation - they just take the current spot and apply that to the NBP curve quoted in pence/therm

* By using the fwd cable curve we should be able to derive a more accurate price on the NBP in dollar per mmbtu terms

In [5]:
# NBP flat prices originally quoted in pence/therm
# Cable fwd curve extraction
def fetch_forward_fx_rates(url, tenor_list):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    data = []

    for tenor in tenor_list:##
        td = soup.find('td', string=lambda s: s and tenor in s)
        if not td:
            print(f"[WARN] {tenor} not found.")
            continue 

        siblings = td.find_next_siblings("td", limit=3)
        if len(siblings) < 3:
            print(f"[WARN] Not enough data for {tenor}.")
            continue

        try:
            bid = float(siblings[0].text.strip())
            ask = float(siblings[1].text.strip())
            mid = float(siblings[2].text.strip())
            data.append({
                "tenor": tenor,
                "bid": round(bid, 4),
                "ask": round(ask, 4),
                "mid": round(mid, 4)
            })
        except Exception as e:
            print(f"[ERROR] FX parsing failed for {tenor}: {e}")
            continue
    
    return pd.DataFrame(data)

In [8]:
# tenors to pull from

tenors = [
    # "One Week", "Two Week", "Three Week", decided to omit the first 3 weeks as it has on average no more than a 2 pip diff to the 1 month contract
    "One Month", "Two Month", "Three Month", "Four Month", "Five Month", "Six Month",
    "Seven Month", "Eight Month", "Nine Month", "Ten Month", "Eleven Month",
    "One Year", "Two Year", "Three Year", "Four Year", "Five Year"
]

fx_url = "https://www.fxempire.com/currencies/gbp-usd/forward-rates"
cable_fwd_data = fetch_forward_fx_rates(fx_url, tenors)
df_fx = cable_fwd_data[["tenor", "mid"]].rename(columns={"mid": "Cable Fwd Rate"})

df_fx

,tenor,Cable Fwd Rate
0,One Month,1.3572
1,Two Month,1.3575
2,Three Month,1.3576
3,Four Month,1.3577
4,Five Month,1.3577
5,Six Month,1.3576
6,Seven Month,1.3575
7,Eight Month,1.3574
8,Nine Month,1.3573
9,Ten Month,1.3571


*Applying the cable fwd rate onto the correct parts of the NBP curve*

In [9]:
## Applying the fwd rate curve to the correct portions of the NBP 
# Step 1: Parse label into date components
df_nbp["parsed_date"] = pd.to_datetime(df_nbp["label"], format="%b'%y")
df_nbp["month"] = df_nbp["parsed_date"].dt.month
df_nbp["year"] = df_nbp["parsed_date"].dt.year

# Step 2: Identify front month and year (first row of NBP)
front_year = df_nbp.iloc[0]["year"]
front_month = df_nbp.iloc[0]["month"]

# Step 3: Map each row to correct FX tenor
def map_fx_tenor(row):
    # calculate how many months ahead this NBP contract is from the front (earliest) contract
    # This assumes df_nbp is already sorted by date and the front_year/front_month were extracted from the first row
    month_diff = (row["year"] - front_year) * 12 + (row["month"] - front_month)

    # for contracts within the first 11 months of the curve (0 to 10 months ahead),
    # use month-specific tenors: "One-month", "Two-month", ... up to "Eleven Month"
    if month_diff < 11:
        return df_fx.iloc[month_diff]["tenor"]

    # For contracts 11 - 22 months ahead, use the 1-year forward FX rate
    elif month_diff < 23:
        return "One Year"
    
    # for contracrts 23 - 34 months ahead, use the 2-year FX rate
    elif month_diff < 35:
        return "Two Year"
    
    # for contracts 35 - 46 months ahead, use the 3-year FX rate
    elif month_diff < 47:
        return "Three Year"
    
    # for contracts 47 - 58 months ahead, use the 4-year FX rate
    elif month_diff < 59:
        return "Four Year"
    
    # for contracts 59 months and beyond, use the 5-year FX rate
    else:
        return "Five Year"

df_nbp["FX Tenor"] = df_nbp.apply(map_fx_tenor, axis=1)

# Step 4: Merge FX rates
df_nbp = df_nbp.merge(df_fx, left_on="FX Tenor", right_on="tenor", how="left")

# Step 5: Convert to $/MMBtu
df_nbp["NBP ($/MMBtu)"] = round(df_nbp["NBP (p/th)"] / 10 * df_nbp["Cable Fwd Rate"], 3)

# Step 6: Keeping the relevant columns
df_nbp = df_nbp[["label", "NBP (p/th)", "Cable Fwd Rate", "NBP ($/MMBtu)"]]
commodity_data["NF"] = df_nbp

In [10]:
# Merge all flat price curves on 'label' (left-to-right)
df_merged = (
    df_ng
    .merge(df_qa, on="label", how="outer")
    .merge(df_ink, on="label", how="outer")
    .merge(df_jkm, on="label", how="outer")
    .merge(df_nbp, on="label", how="outer")  # already includes Cable + $/MMBtu
)

# Chronological sort — keep 'Cash' first if present
df_merged["sort_key"] = df_merged["label"].apply(
    lambda x: pd.Timestamp("1900-01-01") if x == "Cash" else pd.to_datetime(x, format="%b'%y", errors="coerce")
)

df_merged = (
    df_merged
    .sort_values("sort_key")
    .drop(columns="sort_key")
    .reset_index(drop=True)
)

In [ ]:
## Brent conversion to MMbtu from bbls and computing the geographical arbs
# Options to include certain spreads refer below

# Convert Brent to $/MMBtu
df_merged["Brent ($/MMBtu)"] = df_merged["Brent ($/bbl)"] / 5.8

# Geographical Spreads in $/MMBtu
df_merged["JKM v TTF"] = df_merged["JKM ($/MMBtu)"] - df_merged["TTF ($/MMBtu)"]

# Optional spreads: uncomment (#) if needed
# df_merged["HH v TTF"] = df_merged["Henry Hub ($/MMBtu)"] - df_merged["TTF ($/MMBtu)"]
# df_merged["JKM v HH"] = df_merged["JKM ($/MMBtu)"] - df_merged["Henry Hub ($/MMBtu)"]
# df_merged["JKM v NBP"] = df_merged["JKM ($/MMBtu)"] - df_merged["NBP ($/MMBtu)"]
# df_merged["TTF v NBP"] = df_merged["TTF ($/MMBtu)"] - df_merged["NBP ($/MMBtu)"]

df_merged = df_merged[[  
    "label",
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF",                    # core geographical spread

    # Optional spreads for Miguel: Uncomment to include in pricing Sheet
    # Check to ensure the "Optional Spread" computations above are uncommented as well  
    # "HH v TTF",                   # Henry Hub v TTF
    # "JKM v HH",                   # JKM v Henry Hub
    # "JKM v NBP",                  # JKM v NBP
    # "TTF v NBP",                  # TTF v NBP
    # "Brent ($/MMBtu)",            # crude oil in $/MMBtu (Miguel didn't want it)

    "Brent ($/bbl)",                # Brent in original format (Miguel's preference)
    "NBP (p/th)",
    "Cable Fwd Rate"
]]

# Copy final flat price table for export
df_merged_flat_price = df_merged.copy()

# df_merged_flat_price is the df that will
# be exported to sheet 1 in the workbook

# =========================== END of Sheet 1 construction ===========================

**Building of Time spreads, seasonal, Cals, Quarterly (as well as strips)**

i) *M1/M2 spreads across the curve*

In [ ]:
# Dynamically find the last usable row for spreads 

# Remove "Cash" if it's present at the top (M1/M2 logic assumes monthly structure)
df_spreads_base = df_merged[~df_merged["label"].str.lower().eq("cash")].copy()

# Ensure it's sorted properly (should already be, but safety)
df_spreads_base["sort_key"] = pd.to_datetime(df_spreads_base["label"], format="%b'%y", errors="coerce")
df_spreads_base = df_spreads_base.sort_values("sort_key").drop(columns="sort_key").reset_index(drop=True)

# Make sure there's at least 2 rows to do M1/M2
if len(df_spreads_base) < 2:
    raise ValueError("Not enough data to compute time spreads")

# Spread targets
spread_targets = [
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF"
]

# Build spread labels: "Aug/Sep'25", "Sep/Oct'25", etc.
labels_n = df_spreads_base["label"].values[:-1]
labels_np1 = df_spreads_base["label"].values[1:]
spread_labels = [f"{a}/{b}" for a, b in zip(labels_n, labels_np1)]

df_time_spreads = pd.DataFrame()
df_time_spreads["Spread Label"] = spread_labels

# Calculate M1-M2 spreads
for col in spread_targets:
    col_values = df_spreads_base[col].values
    spreads = [
        col_values[i] - col_values[i + 1]
        if pd.notna(col_values[i]) and pd.notna(col_values[i + 1])
        else np.nan
        for i in range(len(col_values) - 1)
    ]
    df_time_spreads[f"{col} Spread"] = spreads

# Optional: round spreads at the end (avoid double rounding so just leave as is)
# df_time_spreads = df_time_spreads.round(3)

# Preview
df_time_spreads.head()

,Spread Label,JKM ($/MMBtu) Spread,TTF ($/MMBtu) Spread,NBP ($/MMBtu) Spread,Henry Hub ($/MMBtu) Spread,JKM v TTF Spread
0,Sep'25/Oct'25,0.410,-0.164,-0.047,-0.099,0.574
1,Oct'25/Nov'25,-0.135,-0.324,-0.873,-0.412,0.189
2,Nov'25/Dec'25,-0.390,-0.171,-0.530,-0.599,-0.219
3,Dec'25/Jan'26,-0.170,-0.067,-0.080,-0.323,-0.103
4,Jan'26/Feb'26,0.030,-0.010,0.002,0.250,0.040


ii) *Summer, Winter legs and the seasonal spreads between*

In [13]:
# Prep: Decompose contract labels
df_merged["contract_date"] = pd.to_datetime(df_merged["label"], format="%b'%y", errors='coerce')
df_merged = df_merged.dropna(subset=["contract_date"]).sort_values("contract_date").reset_index(drop=True)
df_merged["month"] = df_merged["contract_date"].dt.month
df_merged["year"] = df_merged["contract_date"].dt.year

# Define seasonal columns
price_cols = [
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF"
]
df_filtered = df_merged[["label", "contract_date", "month", "year"] + price_cols].copy()

# Seasonal averaging logic
def seasonal_avg(df, season, year):
    if season == "Win":
        mask = ((df["year"] == year) & df["month"].isin([10, 11, 12])) | \
               ((df["year"] == year + 1) & df["month"].isin([1, 2, 3]))
    else:  # Summer
        mask = (df["year"] == year) & df["month"].isin([4, 5, 6, 7, 8, 9])

    df_season = df[mask]
    if df_season.empty:
        return None

    out = {"Spread Label": f"{season}'{str(year)[-2:]}"}
    for col in price_cols:
        out[f"{col} Spread"] = df_season[col].mean(skipna=True)
    return out

In [14]:
# ---- Generate dynamic seasonal ladder in alternating order ----
today = datetime.today()
current_year = today.year
n_winters = 4
n_summers = 5
seasonal_ladder = []

# Always start with the first available Winter
for i in range(n_winters + n_summers):
    if i % 2 == 0:  # Even index → Winter
        y = current_year + (i // 2)
        expiry = datetime(y, 9, 30)
        if today <= expiry:
            win = seasonal_avg(df_filtered, "Win", y)
            if win:
                seasonal_ladder.append(win)
    else:  # Odd index → Summer
        y = current_year + ((i + 1) // 2)
        expiry = datetime(y, 3, 31)
        if today <= expiry:
            sumr = seasonal_avg(df_filtered, "Sum", y)
            if sumr:
                seasonal_ladder.append(sumr)

df_seasons = pd.DataFrame(seasonal_ladder)

In [15]:

# ---- Compute alternating spreads ----
spread_rows = []
for i in range(len(df_seasons) - 1):
    label_1 = df_seasons.iloc[i]["Spread Label"]
    label_2 = df_seasons.iloc[i + 1]["Spread Label"]

    spread_label = f"{label_1} - {label_2}"
    spread_row = {"Spread Label": spread_label}

    for col in price_cols:
        col_spread = f"{col} Spread"
        val_1 = df_seasons.iloc[i][col_spread]
        val_2 = df_seasons.iloc[i + 1][col_spread]
        spread_row[col_spread] = val_1 - val_2 if pd.notnull(val_1) and pd.notnull(val_2) else None

    spread_rows.append(spread_row)

df_season_spreads = pd.DataFrame(spread_rows)

# ---- Final seasonal + spread block ----
df_seasonal_strips = pd.concat([df_seasons, df_season_spreads], ignore_index=True).reset_index(drop=True)
df_seasonal_strips

,Spread Label,JKM ($/MMBtu) Spread,TTF ($/MMBtu) Spread,NBP ($/MMBtu) Spread,Henry Hub ($/MMBtu) Spread,JKM v TTF Spread
0,Win'25,11.903333,11.666667,11.953167,3.640833,0.236667
1,Sum'26,11.355000,11.047667,10.790333,3.656833,0.307333
2,Win'26,11.932500,11.416000,11.789667,4.204833,0.516500
3,Sum'27,10.528333,9.974500,9.868333,3.510167,0.553833
4,Win'27,10.949167,10.346000,10.785500,4.140000,0.603167
5,Sum'28,9.688333,9.079000,9.057500,3.404500,0.609333
6,Win'28,10.202500,9.336667,10.181333,4.025833,0.581667
7,Sum'29,9.135000,NaN,8.825833,3.291333,NaN
8,Win'29,9.602500,NaN,9.991167,3.961000,NaN
9,Win'25 - Sum'26,0.548333,0.619000,1.162833,-0.016000,-0.070667


iii) *Quarterly legs and the spreads between*

In [16]:
# PREP
df = df_merged.copy()
df["contract_date"] = pd.to_datetime(df["label"], format="%b'%y", errors="coerce")
df = df.dropna(subset=["contract_date"]).copy()

df["quarter"] = df["contract_date"].dt.to_period("Q")
df["month"] = df["contract_date"].dt.month
df["year"] = df["contract_date"].dt.year

# EXPIRE QUARTERS LOGIC
# A quarter is expired if its *first month* has settled (settles end of previous month)
today = datetime.today()
expiry_map = {
    1: (12, -1),   # Jan → expires Dec (last day of dec) last year
    4: (3, 0),     # Apr → expires Mar
    7: (6, 0),     # Jul → expires Jun
    10: (9, 0),    # Oct → expires Sep
}

expired_quarters = []
# Introducing a Rolling 4-year horizon from today
cutoff_year = today.year + 4 

for q in df["quarter"].unique():
    q_year = q.year
    first_month = q.start_time.month
    expiry_month, offset_year = expiry_map[first_month]
    
    # Use day=28 as safe fallback for end-of-month
    expiry_date = datetime(q_year + offset_year, expiry_month, 28)
    
    # Drop quarters that have:
    # - Already expired, OR
    # - Have an expiry date beyond our 4-year horizon
    if today > expiry_date or q_year > cutoff_year:
        expired_quarters.append(q)

# Remove expired quarters
df = df[~df["quarter"].isin(expired_quarters)]

In [17]:
# VALID QUARTERS (3 months of data) 
quarter_counts = df.groupby("quarter")["label"].count()
valid_quarters = quarter_counts[quarter_counts == 3].index
df = df[df["quarter"].isin(valid_quarters)]

# QUARTERLY AVERAGES
quarterly_avg = df.groupby("quarter").agg({
    "JKM ($/MMBtu)": "mean",
    "TTF ($/MMBtu)": "mean",
    "NBP ($/MMBtu)": "mean",
    "Henry Hub ($/MMBtu)": "mean",
    "JKM v TTF": "mean"
}).reset_index()

# Format labels: Q1'27, etc.
def format_quarter_label(q): return f"Q{q.quarter}'{str(q.year)[-2:]}"
quarterly_avg["Spread Label"] = quarterly_avg["quarter"].apply(format_quarter_label)
quarterly_avg = quarterly_avg.drop(columns="quarter")

# Reorder & Rename
quarterly_avg = quarterly_avg[[  
    "Spread Label",
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF"
]].rename(columns={
    "JKM ($/MMBtu)": "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu)": "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu)": "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu)": "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF": "JKM v TTF Spread"
})

In [18]:
# Drop rows where all spreads are NaN (to avoid breaking diffs)
quarterly_avg = quarterly_avg.dropna(subset=[
    "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF Spread"
], how="all").reset_index(drop=True)

# ROLLING Q-on-Q SPREADS
rolling_spreads = quarterly_avg.copy()
rolling_spreads["Spread Label"] = (
    quarterly_avg["Spread Label"].shift() + " - " + quarterly_avg["Spread Label"]
)
rolling_spreads.iloc[0, rolling_spreads.columns.get_loc("Spread Label")] = None

for col in [
    "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF Spread"
]:
    rolling_spreads[col] = quarterly_avg[col].shift() - quarterly_avg[col]

rolling_spreads = rolling_spreads.dropna(subset=[
    "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF Spread"
], how="all").reset_index(drop=True)
# FINAL COMBINED OUTPUT
df_quarterly_strips = pd.concat([quarterly_avg, rolling_spreads], ignore_index=True)

df_quarterly_strips

,Spread Label,JKM ($/MMBtu) Spread,TTF ($/MMBtu) Spread,NBP ($/MMBtu) Spread,Henry Hub ($/MMBtu) Spread,JKM v TTF Spread
0,Q4'25,11.735000,11.543000,11.695667,3.362333,0.192000
1,Q1'26,12.071667,11.790333,12.210667,3.919333,0.281333
2,Q2'26,11.258333,11.055333,10.870000,3.490667,0.203000
3,Q3'26,11.451667,11.040000,10.710667,3.823000,0.411667
4,Q4'26,11.780000,11.355333,11.621667,4.151667,0.424667
5,Q1'27,12.085000,11.476667,11.957667,4.258000,0.608333
6,Q2'27,10.530000,10.017000,10.163000,3.362000,0.513000
7,Q3'27,10.526667,9.932000,9.573667,3.658333,0.594667
8,Q4'27,10.778333,10.224000,10.606000,4.024000,0.554333
9,Q1'28,11.120000,10.468000,10.965000,4.256000,0.652000


iv) *Cal legs and the spreads between*

In [19]:
# --- PREP ---
df = df_merged.copy()
df["contract_date"] = pd.to_datetime(df["label"], format="%b'%y", errors="coerce")
df = df.dropna(subset=["contract_date"]).copy()
df["month"] = df["contract_date"].dt.month
df["year"] = df["contract_date"].dt.year

# --- EXPIRE CAL LOGIC ---
today = datetime.today()
cutoff_year = today.year + 5  # Rolling 5-year forward horizon

# Cal year 'YY' expires at end of Dec(YY-1)
valid_years = []
for y in df["year"].unique():
    if today <= datetime(y - 1, 12, 31) and y <= cutoff_year:
        valid_years.append(y)

# Filter to valid years
df = df[df["year"].isin(valid_years)]

# --- CALENDAR AVERAGES ---
cal_avg = df.groupby("year")[[  
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF"
]].mean().round(4).reset_index()

# Format label: Cal 'YY
cal_avg["Spread Label"] = "Cal '" + cal_avg["year"].astype(str).str[-2:]
cal_avg = cal_avg.drop(columns="year")

# Reorder & rename
cal_avg = cal_avg[[  
    "Spread Label",
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF"
]].rename(columns={
    "JKM ($/MMBtu)": "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu)": "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu)": "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu)": "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF": "JKM v TTF Spread"
})

# --- KEEP partial data ---
cal_avg = cal_avg.dropna(
    subset=[
        "JKM ($/MMBtu) Spread",
        "TTF ($/MMBtu) Spread",
        "NBP ($/MMBtu) Spread",
        "Henry Hub ($/MMBtu) Spread",
        "JKM v TTF Spread"
    ],
    how="all"  # drop only rows with ALL NaNs
).reset_index(drop=True)

# --- CALENDAR SPREADS ---
cal_spreads = pd.DataFrame()
cal_spreads["Spread Label"] = (
    cal_avg["Spread Label"].shift() + " - " + cal_avg["Spread Label"]
)

# Compute spreads for each column individually
for col in [
    "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF Spread"
]:
    cal_spreads[col] = cal_avg[col].shift() - cal_avg[col]

# Drop first row (invalid)
cal_spreads = cal_spreads.dropna(subset=["Spread Label"]).reset_index(drop=True)

# --- COMBINE ---
calendar_df = pd.concat([cal_avg, cal_spreads], ignore_index=True)
calendar_df

,Spread Label,JKM ($/MMBtu) Spread,TTF ($/MMBtu) Spread,NBP ($/MMBtu) Spread,Henry Hub ($/MMBtu) Spread,JKM v TTF Spread
0,Cal '26,11.6404,11.3103,11.3533,3.8462,0.3302
1,Cal '27,10.9800,10.4124,10.5751,3.8256,0.5676
2,Cal '28,10.1038,9.4907,9.7728,3.7494,0.6131
3,Cal '29,9.5271,NaN,9.3965,3.6401,NaN
4,Cal '30,8.9517,NaN,9.4676,3.4994,NaN
5,Cal '26 - Cal '27,0.6604,0.8979,0.7782,0.0206,-0.2374
6,Cal '27 - Cal '28,0.8762,0.9217,0.8023,0.0762,-0.0455
7,Cal '28 - Cal '29,0.5767,NaN,0.3763,0.1093,NaN
8,Cal '29 - Cal '30,0.5754,NaN,-0.0711,0.1407,NaN


**Bringing all the spreads together and organising in the ideal manner**

In [20]:
## Bringing  all together for sheet 2
 # Define consistent column order
columns_order = [
    "Spread Label",
    "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF Spread"
]

# Ensure all DataFrames have the same columns
def align_columns_safe(df, name):
    try:
        return df.reindex(columns=columns_order)
    except Exception as e:
        print(f"[ERROR] Align failed for {name}: {e}")
        return pd.DataFrame(columns=columns_order)

df_time_spreads = align_columns_safe(df_time_spreads, "df_time_spreads")
quarterly_df = align_columns_safe(df_quarterly_strips, "quarterly_df")
df_season_output = align_columns_safe(df_seasonal_strips, "df_season_output")
calendar_df = align_columns_safe(calendar_df, "calendar_df")

# Create a single blank row with NaNs
blank_row = pd.DataFrame([[""] + [None] * (len(columns_order) - 1)], columns=columns_order)

# Concatenate with blank rows in between
df_all_spreads = pd.concat([
    df_time_spreads,
    blank_row,
    quarterly_df,
    blank_row,
    df_season_output,
    blank_row,
    calendar_df
], ignore_index=True) # this can go in sheet 2

# =========================== END of Sheet 2 construction ===========================

C:\Users\aran.willetts\AppData\Local\Temp\ipykernel_19580\4102758108.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all_spreads = pd.concat([


**Exporting to workbook - the product that we made**

* Allocating the dfs to the respective sheets in the workbook
* Using the excel writer to freeze certain planes, align the cells and obtain ideal formatting 

In [ ]:
# Create filename
today_str = datetime.today().strftime("%Y-%m-%d")
filename = f"LNG_Pricing_Sheet_{today_str}.xlsx"

# --- Round ONLY sheet 2 ---
for col in df_all_spreads.columns:
    if "Spread" in col and df_all_spreads[col].dtype.kind in "fc":
        df_all_spreads[col] = df_all_spreads[col].round(3)

# --- Export to Excel ---
with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
    # Write both DataFrames
    df_merged_flat_price.to_excel(writer, sheet_name="Flat Prices", index=False)
    df_all_spreads.to_excel(writer, sheet_name="Spread Summary", index=False)

    # Access workbook and worksheets
    workbook = writer.book
    sheet1 = writer.sheets["Flat Prices"]
    sheet2 = writer.sheets["Spread Summary"]

    # Define formats
    format_2dp = workbook.add_format({"align": "center", "valign": "vcenter", "num_format": "0.00"})
    format_3dp = workbook.add_format({"align": "center", "valign": "vcenter", "num_format": "0.000"})
    format_4dp = workbook.add_format({"align": "center", "valign": "vcenter", "num_format": "0.0000"})
    format_default = workbook.add_format({"align": "center", "valign": "vcenter"})

    # --- Sheet 1 formatting ---
    for idx, col in enumerate(df_merged_flat_price.columns):
        max_len = max(len(str(col)), df_merged_flat_price[col].astype(str).map(len).max()) + 2

        if col == "Brent ($/bbl)" or col == "NBP (p/th)":
            fmt = format_2dp
        elif col == "Cable Fwd Rate":
            fmt = format_4dp
        elif df_merged_flat_price[col].dtype.kind in "fc":
            fmt = format_3dp
        else:
            fmt = format_default

        sheet1.set_column(idx, idx, max_len, fmt)

    # --- Sheet 2 formatting (all 3dp) ---
    for idx, col in enumerate(df_all_spreads.columns):
        max_len = max(len(str(col)), df_all_spreads[col].astype(str).map(len).max()) + 2
        sheet2.set_column(idx, idx, max_len, format_3dp)

    # Freeze top row on both
    sheet1.freeze_panes(1, 0)
    sheet2.freeze_panes(1, 0)

# Serves as a checker in the original ipynb
print(f"Exported to: {filename}")

**Delivery (to traders)**

* Note: that the yml file should contain the password key to send to ur email address not in the script but for the sake of learning just use: 

- `from_email="aran.willetts@gmail.com"`
- `smtp_server="smtp.gmail.com"`
- `smtp_port=465`
- `login="aran.willetts@gmail.com"`
- `password=os.environ["EMAIL_PASSWORD"]`
- `EMAIL_PASSWORD = "plfrmqdnsvxfzgeo"` 

* Note: not the actual email password, but a code provided by google to access the server externally

In [ ]:
def send_email_with_attachment(
    filename,
    subject,
    body,
    to_email,
    from_email,
    smtp_server,
    smtp_port,
    login,
    password,
    cc=None,
    bcc=None
):

    # Create the email
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = ', '.join(to_email)
    
    if cc:
        msg['Cc'] = ', '.join(cc)
    
    msg.set_content(body)

    # Attach the Excel file
    with open(filename, 'rb') as f:
        file_data = f.read()
        file_name = f.name

    msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)

    # Combine all recipients
    all_recipients = to_email + (cc if cc else []) + (bcc if bcc else [])

    # Send the email
    with smtplib.SMTP_SSL(smtp_server, smtp_port) as smtp:
        smtp.login(login, password)
        smtp.send_message(msg, to_addrs=all_recipients)

# Email config
send_email_with_attachment(
    filename=filename,
    subject=f"LNG Pricing Sheet - {today_str}",
    body="Hi Miguel, \n\nPlease find attached the latest LNG pricing sheet.",
    to_email=["Miguel.Arroyo@irh.ae"],  # Primary recipients
    cc=["Lara.Peel@irh.ae", "aran.willetts@irh.ae", "vedant.bundellu@irh.ae"],        # Optional CC list
    #bcc=["hidden1@irh.ae", "hidden2@irh.ae"],  # Commented out for now
    from_email="aran.willetts@gmail.com",
    smtp_server="smtp.gmail.com",
    smtp_port=465,
    login="aran.willetts@gmail.com",
    password=os.environ["EMAIL_PASSWORD"]
)